# Code4Earth Issue 25 - Tesserugged, Downscaling of ERA5 to CERRA using a UNET and different input data types

This notebook provides documentation on the UNET downscaling approach for Code4Earth Issue 25 and guides through the different UNET versions.

Different versions and generators, depending on the hardware capabilities, were programmed.

As for the input experiments, also different data sets were planned to be tested and the code was prepared accordingly.

## Data
Two different input data types can be used: min max normalized and the residuals used in SAMOS.

### Minmax-Normalized data
The classical **minmax-normalized** CERRA/ERA5 data, normalized with a per pixel minmax approach. This uses the provided, non-normalized data and normalizes them "on-the-fly" within the generator.

The "climate" field data are generated using the training period 1985 - 2016, leaving the two validation years 2017 & 2018 and the test years 2019 - 2021 out. The following CDO commands are used:

```shell
cdo -timXXX -mergetime t2m_era5_198* t2m_era5_199* t2m_era5_200* t2m_era5_2010* t2m_era5_2011* t2m_era5_2012* t2m_era5_2013* t2m_era5_2014* t2m_era5_2015* t2m_era5_2016* /ssea/SSEA/C4E/MODELS/UNET/DATA/CLIMS/t2m_era5_1985-2016_overall_XXX.nc
```

The drawback is that we possibly leave some extremes out for standardizing. However, for test purposes this should be fine anyway.

### Residual approach
Based on the idea of SAMOS (and other more recent but also bit older publications, see the git repo for more info) is to use per-grid point standardized anomalies with fitting applied to account for seasonalities. This enables a generalized model as we use anomalies with respect to the local (grid point) climatology. Here, too, we need to perform a normalization as the anomalies can be larget then +-1.

### Static data 
Additionally to the to be downscaled parameter, static information is used. In this approach, the topography and land-sea mask information is included. As input, highres+lowres topography and lsm are used. Based on the idea of Sha et al. (2020), also the highres topography is used as second target field.

## The UNET downscaling
The code is structured as:

```
main_unet_downscaling.py
train.py
predict.py
custom_loss_functions.py
unets.py
generator.py
utils.py
```

where ```main_unet_downscaling.py``` is used for both training and prediction.

The run the code one needs to give it several arguments while calling:

```shell
python main_unet_downscaling.py --model USha --mode training --modelname MODEL/UNET_v0_hyper_REDISUALS_MSE_USha_wZ_00 --lossfunction MSE --pathtotarget DATA/FILELISTS/target_unet_residuals_00_temp.csv --pathtofeatures DATA/FILELISTS/features_unet_residuals_00_temp.csv --datasource residuals --z_branch True --normalize True --leadtime 00 
```

for training with the following arguments defining what type of model is used, which mode, which name, loss function, location of files, etc.:
```python 
    parser.add_argument('-model',"--model", help="choose model architecture from: Ularge Usmall Uswindow USha", type=str, required=True)
    parser.add_argument('-mode',"--mode", help="choose mode from: training prediction", type=str, required=True)
    parser.add_argument('-modelname',"--modelname", help="the name to save the model to/load the model from", type=str, required=True)
    parser.add_argument('-loss',"--lossfunction", help="the loss function to use: MSE, MAE, IWMSE", type=str, default='MSE')
    parser.add_argument('-target',"--pathtotarget", help="the path to the target csv filelist", type=str)
    parser.add_argument('-features',"--pathtofeatures", help="the path to the features csv filelist", type=str)
    parser.add_argument('-datasource',"--datasource", help="What data source we are using: minmaxnorm, residuals", type=str)
    parser.add_argument('-z_branch',"--z_branch", help="Are we prediction high res topography? True, False", type=str)
    parser.add_argument('-normalize',"--normalize", help="Do we need to normalize the data in the batch generator? True, False",type=str)
    parser.add_argument('-ldtime',"--leadtime", help="Which leadtime we are working on",type=int)
```

Mind, there are still several parts in the code that are, right now, hardcoded like links to the topography and lsm files, the climate-files for normalisation when using the *datasource minmaxnorm*, etc. Also, depending on that kind of datasource you have, the batch size right now is hardcoded (different file structure). However, we tried to minimize those parts and document it in the code.

### UNET versions
The four different UNET versions available are based on the capability of the hardware and other ideas. Names below are the ones which are used in running the script:

- **Ularge**: this unet is made for the whole input grid of CERRA. Still in the code but not used here / not tested.
- **Usmall**: uses the with the spanish team agreed on grid sizes (160,240). Own implementation which could still use some hyperparameter tuning.
- **USha**: based on the idea of Sha et al. (2020) and the implementation used in the ECMWF training course in May 2022. Was adpated during the course (# layers changed, normalization of layers changed, etc.), adapted version used here but could most likely still be improved.
- **Uswindow**: for not so big hardware this model uses running windows of (64,64) grids with (4,4) overlapping grid points. In case the chunked domain is smaller, the generator extends the chunk so that it fits with the (64,64) grid.

Depending on the UNET version also the generator version is set in the code. Inside the generators, right now, the data is normalized on the fly, and if needed, the features are interpolated to the target grid and any NaNs are filled.

Right now, we are using ```he_normalisation``` for initialisation as that proved to work. But, play around and see what works best for you.

### Loss functions

Three loss functions are available right now, others are in the ```custom_loss_functions.py``` but not used.

- MSE: inbuild MSE function, depending if you use tensorflow 2 or tensorflow 1 it calls the respective functions
- MAE: inbuild MAE function, depending if you use tensorflow 2 or tensorflow 1 it calls the respective functions
- InvMSE: defined in ```custom_loss_functions.py```, it calculates on the fly the inversely weighted MSE

### Structure
In **MODEL** right now we are saving the different trained models, DATA contains the filelist (**DATA/FILELISTS**) used for training and the climate files for normalisation are in **DATA/CLIMS**.